# PyPika Tutorial: Slicing, Averaging, and Preserving Order

This notebook will demonstrate how to use PyPika to create SQL queries that involve:
- Basic selection and filtering (slicing).
- Aggregation (averaging).
- Combining selection, averaging, and ensuring the order is maintained for other functions that depend on ordering.

Let's get started by installing and importing the necessary packages.

## Step 1: Installation and Imports

First, ensure that you have PyPika installed.
```bash
!pip install pypika
```

Now, import PyPika and any other required packages:

In [12]:
from pypika import Table, Field, functions as fn
import sqlite3  # To simulate the database
import pandas as pd
import numpy as np

For demonstration purposes, we will create a small SQLite database with a table named "ds1" and populate it with some sample data.


## Step 2: Creating a Sample Database

For demonstration purposes, we will create a small SQLite database with a table named "ds1" and populate it with a more realistic 3D dataset.

The dataset will represent a grid mesh where `x` and `y` span from -1 to 1, and `z` is a Gaussian response as a function of `x` and `y` with added noise. We will also have `a` representing 10 copies with different noise levels simulating repeat measurements, and `b` representing two states that flip the sign of `z`.


In [13]:
# Establish a connection to an in-memory SQLite database
connection = sqlite3.connect(':memory:')
cursor = connection.cursor()

# Create the "ds1" table
cursor.execute('''
CREATE TABLE ds1 (
    x REAL,
    y REAL,
    z REAL,
    a INTEGER,
    b INTEGER
);
''')

# Generate sample data
x_vals = np.linspace(-1, 1, 5)
y_vals = np.linspace(-1, 1, 5)
a_vals = range(1, 11)  # 10 copies with different noise
b_vals = [1, -1]  # Two states flipping the sign of z

sample_data = []
for x in x_vals:
    for y in y_vals:
        base_z = np.exp(-(x**2 + y**2))  # Gaussian response
        for a in a_vals:
            noise = np.random.normal(0, 0.1)  # Adding noise
            for b in b_vals:
                z = base_z * b + noise
                sample_data.append((x, y, z, a, b))

# Insert the sample data into "ds1"
cursor.executemany('INSERT INTO ds1 VALUES (?, ?, ?, ?, ?)', sample_data)
connection.commit()

## Step 3: Slicing and Averaging Using PyPika

Now, let's use PyPika to build SQL queries that allow us to perform slicing (selecting rows based on conditions) and averaging.



### Example 1: Simple Selection
We want to select columns `x`, `y`, `z`, `a`, and `b` from `ds1` where `a == 1`.

In [10]:
# Define the table
ds1 = Table('ds1')

# Build the query
query = ds1.select(ds1.x, ds1.y, ds1.z, ds1.a).where(ds1.a == 1)
print(query)
# Execute the query
result_df = pd.read_sql_query(str(query), connection)
print("\nSimple Selection Result:")
print(result_df)

SELECT "x","y","z","a" FROM "ds1" WHERE "a"=1

Simple Selection Result:
   x  y   z  a
0  1  1  10  1
1  1  2  20  1
2  2  1  40  1
3  2  2  50  1


### Example 2: Aggregating with Averaging
We want to select columns `x`, `y`, and calculate the average of `z` where `a == 1`. The result should be grouped by `x` and `y`.


In [19]:
# Define the table
ds1 = Table('ds1')

# Build the query
query = ds1.select(ds1.x, ds1.y, ds1.z, ds1.a, ds1.b).where(ds1.a == 1)
print(query)
# Execute the query
result_df = pd.read_sql_query(str(query), connection)
print("Simple Selection Result:")
print(result_df)

SELECT "x","y","z","a","b" FROM "ds1" WHERE "a"=1
Simple Selection Result:
      x    y         z  a  b
0  -1.0 -1.0  0.119441  1  1
1  -1.0 -1.0 -0.151230  1 -1
2  -1.0 -0.5  0.289396  1  1
3  -1.0 -0.5 -0.283613  1 -1
4  -1.0  0.0  0.211500  1  1
5  -1.0  0.0 -0.524258  1 -1
6  -1.0  0.5  0.314464  1  1
7  -1.0  0.5 -0.258546  1 -1
8  -1.0  1.0 -0.028888  1  1
9  -1.0  1.0 -0.299559  1 -1
10 -0.5 -1.0  0.308932  1  1
11 -0.5 -1.0 -0.264078  1 -1
12 -0.5 -0.5  0.667427  1  1
13 -0.5 -0.5 -0.545634  1 -1
14 -0.5  0.0  0.822982  1  1
15 -0.5  0.0 -0.734620  1 -1
16 -0.5  0.5  0.596444  1  1
17 -0.5  0.5 -0.616617  1 -1
18 -0.5  1.0  0.294913  1  1
19 -0.5  1.0 -0.278097  1 -1
20  0.0 -1.0  0.456386  1  1
21  0.0 -1.0 -0.279373  1 -1
22  0.0 -0.5  0.693570  1  1
23  0.0 -0.5 -0.864031  1 -1
24  0.0  0.0  0.991673  1  1
25  0.0  0.0 -1.008327  1 -1
26  0.0  0.5  0.882755  1  1
27  0.0  0.5 -0.674846  1 -1
28  0.0  1.0  0.346760  1  1
29  0.0  1.0 -0.388999  1 -1
30  0.5 -1.0  0.219091  1 

### Example 2: Aggregating with Averaging
We want to select columns `x`, `y`, and calculate the average of `z` where `a == 1`. The result should be grouped by `x` and `y`.

In [20]:
# Build the query for aggregation
query = (
    ds1
    .select(ds1.x, ds1.y, fn.Avg(ds1.z).as_('avg_z'))
    .where(ds1.a == 1)
    .groupby(ds1.x, ds1.y)
    .orderby(ds1.x, ds1.y)
)
print(query)

# Execute the query
result_df = pd.read_sql_query(str(query), connection)
print("Aggregation with Averaging Result:")
print(result_df)

SELECT "x","y",AVG("z") "avg_z" FROM "ds1" WHERE "a"=1 GROUP BY "x","y" ORDER BY "x","y"
Aggregation with Averaging Result:
      x    y     avg_z
0  -1.0 -1.0 -0.015895
1  -1.0 -0.5  0.002892
2  -1.0  0.0 -0.156379
3  -1.0  0.5  0.027959
4  -1.0  1.0 -0.164224
5  -0.5 -1.0  0.022427
6  -0.5 -0.5  0.060896
7  -0.5  0.0  0.044181
8  -0.5  0.5 -0.010087
9  -0.5  1.0  0.008408
10  0.0 -1.0  0.088506
11  0.0 -0.5 -0.085231
12  0.0  0.0 -0.008327
13  0.0  0.5  0.103954
14  0.0  1.0 -0.021119
15  0.5 -1.0 -0.067414
16  0.5 -0.5  0.254255
17  0.5  0.0 -0.138316
18  0.5  0.5 -0.025209
19  0.5  1.0  0.053586
20  1.0 -1.0 -0.014201
21  1.0 -0.5 -0.040890
22  1.0  0.0  0.018729
23  1.0  0.5 -0.101062
24  1.0  1.0  0.027303



### Example 3: Combining Slicing, Averaging, and Preserving Order
We want to make sure that when we select `x`, `y`, and `avg(z)` where `a == 1`, the result preserves the order of `x` and `y` so that other functions depending on the order will still work.


In this example, we ensure the use of `.orderby()` to preserve the order of `x` and `y` in the final result. This ensures that if you pass the resulting DataFrame to any function that relies on the order of `x` and `y`, it will work correctly.


In [21]:
# Build the combined query
query = (
    ds1
    .select(ds1.x, ds1.y, fn.Avg(ds1.z).as_('avg_z'))
    .where(ds1.a == 1)
    .groupby(ds1.x, ds1.y)
    .orderby(ds1.x, ds1.y)  # Ensure the ordering is maintained
)

print(query)
# Execute the query
result_df = pd.read_sql_query(str(query), connection)
print("Combined Selection and Averaging with Order Preserved:")
print(result_df)


# Verify that the ordering is maintained for further processing
print("Is the DataFrame sorted by ['x', 'y']?")
print(result_df.equals(result_df.sort_values(by=['x', 'y'])))

SELECT "x","y",AVG("z") "avg_z" FROM "ds1" WHERE "a"=1 GROUP BY "x","y" ORDER BY "x","y"
Combined Selection and Averaging with Order Preserved:
      x    y     avg_z
0  -1.0 -1.0 -0.015895
1  -1.0 -0.5  0.002892
2  -1.0  0.0 -0.156379
3  -1.0  0.5  0.027959
4  -1.0  1.0 -0.164224
5  -0.5 -1.0  0.022427
6  -0.5 -0.5  0.060896
7  -0.5  0.0  0.044181
8  -0.5  0.5 -0.010087
9  -0.5  1.0  0.008408
10  0.0 -1.0  0.088506
11  0.0 -0.5 -0.085231
12  0.0  0.0 -0.008327
13  0.0  0.5  0.103954
14  0.0  1.0 -0.021119
15  0.5 -1.0 -0.067414
16  0.5 -0.5  0.254255
17  0.5  0.0 -0.138316
18  0.5  0.5 -0.025209
19  0.5  1.0  0.053586
20  1.0 -1.0 -0.014201
21  1.0 -0.5 -0.040890
22  1.0  0.0  0.018729
23  1.0  0.5 -0.101062
24  1.0  1.0  0.027303
Is the DataFrame sorted by ['x', 'y']?
True
